# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f94b038d130>
├── 'a' --> tensor([[-0.8975,  1.0761,  0.8069],
│                   [ 1.7547,  0.7596, -0.7669]])
└── 'x' --> <FastTreeValue 0x7f94b031ab50>
    └── 'c' --> tensor([[ 0.2738,  0.4156,  1.9978,  0.3955],
                        [ 0.4768, -2.0624,  0.8400, -0.3789],
                        [-0.1468, -0.7355, -0.4568,  0.0985]])

In [4]:
t.a

tensor([[-0.8975,  1.0761,  0.8069],
        [ 1.7547,  0.7596, -0.7669]])

In [5]:
%timeit t.a

58.2 ns ± 0.0342 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f94b038d130>
├── 'a' --> tensor([[-0.7726,  2.2853,  0.0190],
│                   [ 0.8829,  1.5787, -1.4346]])
└── 'x' --> <FastTreeValue 0x7f94b031ab50>
    └── 'c' --> tensor([[ 0.2738,  0.4156,  1.9978,  0.3955],
                        [ 0.4768, -2.0624,  0.8400, -0.3789],
                        [-0.1468, -0.7355, -0.4568,  0.0985]])

In [7]:
%timeit t.a = new_value

62.3 ns ± 0.0219 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.8975,  1.0761,  0.8069],
               [ 1.7547,  0.7596, -0.7669]]),
    x: Batch(
           c: tensor([[ 0.2738,  0.4156,  1.9978,  0.3955],
                      [ 0.4768, -2.0624,  0.8400, -0.3789],
                      [-0.1468, -0.7355, -0.4568,  0.0985]]),
       ),
)

In [10]:
b.a

tensor([[-0.8975,  1.0761,  0.8069],
        [ 1.7547,  0.7596, -0.7669]])

In [11]:
%timeit b.a

58 ns ± 0.0198 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7665,  0.2960, -1.1857],
               [ 0.7212, -1.0982,  0.1820]]),
    x: Batch(
           c: tensor([[ 0.2738,  0.4156,  1.9978,  0.3955],
                      [ 0.4768, -2.0624,  0.8400, -0.3789],
                      [-0.1468, -0.7355, -0.4568,  0.0985]]),
       ),
)

In [13]:
%timeit b.a = new_value

483 ns ± 0.747 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

915 ns ± 6.96 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 16.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 442 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 1.87 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f94001ff880>
├── 'a' --> tensor([[[-0.8975,  1.0761,  0.8069],
│                    [ 1.7547,  0.7596, -0.7669]],
│           
│                   [[-0.8975,  1.0761,  0.8069],
│                    [ 1.7547,  0.7596, -0.7669]],
│           
│                   [[-0.8975,  1.0761,  0.8069],
│                    [ 1.7547,  0.7596, -0.7669]],
│           
│                   [[-0.8975,  1.0761,  0.8069],
│                    [ 1.7547,  0.7596, -0.7669]],
│           
│                   [[-0.8975,  1.0761,  0.8069],
│                    [ 1.7547,  0.7596, -0.7669]],
│           
│                   [[-0.8975,  1.0761,  0.8069],
│                    [ 1.7547,  0.7596, -0.7669]],
│           
│                   [[-0.8975,  1.0761,  0.8069],
│                    [ 1.7547,  0.7596, -0.7669]],
│           
│                   [[-0.8975,  1.0761,  0.8069],
│                    [ 1.7547,  0.7596, -0.7669]]])
└── 'x' --> <FastTreeValue 0x7f94b03140d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.7 µs ± 74.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f940acca160>
├── 'a' --> tensor([[-0.8975,  1.0761,  0.8069],
│                   [ 1.7547,  0.7596, -0.7669],
│                   [-0.8975,  1.0761,  0.8069],
│                   [ 1.7547,  0.7596, -0.7669],
│                   [-0.8975,  1.0761,  0.8069],
│                   [ 1.7547,  0.7596, -0.7669],
│                   [-0.8975,  1.0761,  0.8069],
│                   [ 1.7547,  0.7596, -0.7669],
│                   [-0.8975,  1.0761,  0.8069],
│                   [ 1.7547,  0.7596, -0.7669],
│                   [-0.8975,  1.0761,  0.8069],
│                   [ 1.7547,  0.7596, -0.7669],
│                   [-0.8975,  1.0761,  0.8069],
│                   [ 1.7547,  0.7596, -0.7669],
│                   [-0.8975,  1.0761,  0.8069],
│                   [ 1.7547,  0.7596, -0.7669]])
└── 'x' --> <FastTreeValue 0x7f93f9bd66d0>
    └── 'c' --> tensor([[ 0.2738,  0.4156,  1.9978,  0.3955],
                        [ 0.4768, -2.0624,  0.8400, -0.3789],
                 

In [23]:
%timeit t_cat(trees)

29.7 µs ± 39.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

69.3 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.2738,  0.4156,  1.9978,  0.3955],
                       [ 0.4768, -2.0624,  0.8400, -0.3789],
                       [-0.1468, -0.7355, -0.4568,  0.0985]],
              
                      [[ 0.2738,  0.4156,  1.9978,  0.3955],
                       [ 0.4768, -2.0624,  0.8400, -0.3789],
                       [-0.1468, -0.7355, -0.4568,  0.0985]],
              
                      [[ 0.2738,  0.4156,  1.9978,  0.3955],
                       [ 0.4768, -2.0624,  0.8400, -0.3789],
                       [-0.1468, -0.7355, -0.4568,  0.0985]],
              
                      [[ 0.2738,  0.4156,  1.9978,  0.3955],
                       [ 0.4768, -2.0624,  0.8400, -0.3789],
                       [-0.1468, -0.7355, -0.4568,  0.0985]],
              
                      [[ 0.2738,  0.4156,  1.9978,  0.3955],
                       [ 0.4768, -2.0624,  0.8400, -0.3789],
                       [-0.1468, -0.7355, -0.4568,  0.0985]],

In [26]:
%timeit Batch.stack(batches)

77.8 µs ± 218 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.2738,  0.4156,  1.9978,  0.3955],
                      [ 0.4768, -2.0624,  0.8400, -0.3789],
                      [-0.1468, -0.7355, -0.4568,  0.0985],
                      [ 0.2738,  0.4156,  1.9978,  0.3955],
                      [ 0.4768, -2.0624,  0.8400, -0.3789],
                      [-0.1468, -0.7355, -0.4568,  0.0985],
                      [ 0.2738,  0.4156,  1.9978,  0.3955],
                      [ 0.4768, -2.0624,  0.8400, -0.3789],
                      [-0.1468, -0.7355, -0.4568,  0.0985],
                      [ 0.2738,  0.4156,  1.9978,  0.3955],
                      [ 0.4768, -2.0624,  0.8400, -0.3789],
                      [-0.1468, -0.7355, -0.4568,  0.0985],
                      [ 0.2738,  0.4156,  1.9978,  0.3955],
                      [ 0.4768, -2.0624,  0.8400, -0.3789],
                      [-0.1468, -0.7355, -0.4568,  0.0985],
                      [ 0.2738,  0.4156,  1.9978,  0.3955],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 356 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

306 µs ± 725 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
